# Day 1a - From Prompt to Action

This notebook demonstrates how to build an **AI Agent** that can understand natural-language instructions and perform actions autonomously using the **Google Agent Development Kit (ADK)** and **Gemini API**.

Based on the Kaggle course: *5 Days of AI Agents with Google*

## Setup and Installation

First, let's install the required dependencies:

In [ ]:
# Install required packages
!pip install google-adk google-generativeai python-dotenv

## 1. Helper Functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [ ]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="background-color: #e8f4f8; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
        <h3 style="margin-top: 0; color: #1976D2;">ADK Proxy URL</h3>
        <p><strong>URL:</strong> <a href="{url}" target="_blank">{url}</a></p>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

## 2. Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [ ]:
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these HTTP errors
)

print("✅ Retry configuration set.")

## 3. Import ADK Components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

## 4. What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

**Prompt → LLM → Text**

An AI Agent takes this one step further. An agent can **think**, **take actions**, and **observe** the results of those actions to give you a better answer.

**Prompt → Agent → Thought → Action → Observation → Final Answer**

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

## 5. Configure Environment Variables

Set your **Gemini API key**. You can get one from [Google AI Studio](https://makersuite.google.com/app/apikey).

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set your API key here or in a .env file
# os.environ["GEMINI_API_KEY"] = "your-api-key-here"

# Verify API key is set
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    print("⚠️ Please set your GEMINI_API_KEY environment variable")
else:
    print("✅ API key configured")

## 6. Define Your Agent

Now, let's build our agent. We'll configure an Agent by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the [documentation related to agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [ ]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

## 7. Run Your Agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [Runner](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

### a. Create an InMemoryRunner

Create an InMemoryRunner and tell it to use our `root_agent`:

In [ ]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

👉 **Note:** We are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the [documentation related to runtime in ADK](https://google.github.io/adk-docs/runtime/).

### b. Send a Query

Now you can call the `.run_debug()` method to send our prompt and get an answer.

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [ ]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)

print("\n" + "="*70)
print("Agent Response:")
print("="*70)
print(response)

## 8. Try More Examples

Now that you have a working agent, try asking it different questions!

In [ ]:
# Example: Ask about current events
response = await runner.run_debug(
    "What are the latest developments in AI agents?"
)
print(response)

In [ ]:
# Example: Ask the agent to help with a task
response = await runner.run_debug(
    "Create a short poem about AI Agents learning from humans."
)
print(response)

## 9. Using the Reusable Agent Module

You can also use the agent implementation from `src/agent.py` for a more streamlined experience:

In [ ]:
from src.agent import create_agent, create_runner, run_agent

# Create an agent with default settings
my_agent = create_agent()
print(f"✅ Agent '{my_agent.name}' created")

# Run a query
response = await run_agent(
    "Explain how AI agents differ from traditional chatbots.",
    my_agent
)
print("\nResponse:", response)

## Next Steps

You can extend this agent by:
- 🧰 Adding memory or context chaining for multi-step reasoning
- 🔗 Integrating external tools (search, data APIs, or code execution)
- 🎨 Building a web interface to interact with your agent visually
- 🧩 Creating custom tools for domain-specific tasks

Explore the `src/agent.py` file to see the implementation details and customize the agent for your needs!